In [ ]:
# ==========================================
# CELL 1: ENVIRONMENT SETUP (COLAB NATIVE - V4)
# ==========================================

print("🔧 Setting up production environment...")

# STRATEGY: Use Colab's pre-installed packages (don't fight the system)
# Colab already has: numpy 2.x, scipy 1.13, pandas 2.x, sklearn 1.x

# Only install what's missing or needs upgrade
!pip install -q yfinance xgboost==2.0.3 lightgbm==4.1.0 joblib python-dotenv

# Optional: pandas-ta (fallback to manual TA if fails)
!pip install -q pandas-ta 2>/dev/null || echo "⚠️ pandas-ta skipped (will use manual TA)"

print("✅ Packages installed - importing...")

# Core imports
import yfinance as yf
import pandas as pd
import numpy as np
import xgboost as xgb
import scipy
import sklearn
import joblib
import requests
import warnings
import json
import logging
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from typing import List, Dict, Tuple
import os

# Try pandas-ta (fallback to manual TA if fails)
try:
    import pandas_ta as ta
    HAS_PANDAS_TA = True
except:
    HAS_PANDAS_TA = False
    print("⚠️ pandas-ta not available, using manual TA calculations")

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('Alpha76')

# GPU Check
try:
    import torch
    if torch.cuda.is_available():
        print(f"🔥 GPU: {torch.cuda.get_device_name(0)}")
        print(f"🔥 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    else:
        print("⚠️ No GPU - training will use CPU (slower)")
except ImportError:
    print("⚠️ PyTorch not found - XGBoost will try to use GPU directly")

print("\n" + "="*70)
print("🏆 HEDGEFUND KILLER - PRODUCTION TRAINER")
print("="*70)
print(f"✅ NumPy: {np.__version__}")
print(f"✅ Scipy: {scipy.__version__}")
print(f"✅ Sklearn: {sklearn.__version__}")
print(f"✅ XGBoost: {xgb.__version__}")
print(f"✅ Pandas: {pd.__version__}")
print(f"✅ YFinance: {yf.__version__}")
print(f"✅ Pandas-TA: {'Available' if HAS_PANDAS_TA else 'Manual mode'}")
print("="*70)
print("\n💡 Using Colab's native package versions (no conflicts)")
print("   This is the SMART way - don't fight the system!\n")

## 🚜 MODULE 1: DATA HARVESTER
Production-grade data collection with quality checks and validation.

In [ ]:
# ==========================================
# MODULE 1: DATA HARVESTER
# ==========================================

class DataHarvester:
    """Download and validate market data"""
    
    def __init__(self, tickers: List[str], period: str = "2y", interval: str = "1h"):
        self.tickers = tickers
        self.period = period
        self.interval = interval
        
    def harvest(self) -> pd.DataFrame:
        """Download with quality checks"""
        logger.info(f"🚜 Harvesting {len(self.tickers)} tickers...")
        
        data = yf.download(
            self.tickers,
            period=self.period,
            interval=self.interval,
            group_by='ticker',
            auto_adjust=True,
            progress=True,
            threads=True
        )
        
        dfs = []
        warnings_list = []
        
        for ticker in self.tickers:
            try:
                df = data[ticker].copy() if len(self.tickers) > 1 else data.copy()
                
                if df.empty or df['Volume'].sum() == 0:
                    warnings_list.append(f"{ticker}: No data")
                    continue
                
                df['ticker'] = ticker
                df.reset_index(inplace=True)
                dfs.append(df)
                
                logger.info(f"  ✅ {ticker}: {len(df)} bars")
                
            except Exception as e:
                warnings_list.append(f"{ticker}: {str(e)}")
        
        if warnings_list:
            logger.warning(f"\n⚠️ Warnings:\n" + "\n".join([f"  - {w}" for w in warnings_list]))
        
        result = pd.concat(dfs, ignore_index=True)
        logger.info(f"\n✅ Total: {len(result):,} rows from {len(dfs)} tickers")
        
        return result

# ===================================================================
# 🏆 HEDGEFUND KILLER UNIVERSE - YOUR PERSONAL WATCHLIST (100+ TICKERS)
# ===================================================================

# 💼 YOUR ROBINHOOD PORTFOLIO (Current Holdings - Track Daily)
PORTFOLIO = [
    'KDK', 'TLRY', 'SERV', 'RR', 'HOOD', 'LYFT', 'UBER',
    'MVST', 'APLD', 'SGBX', 'LUNR', 'IONQ', 'XBIO', 'ASTS'
]

# 🔥 PERPLEXITY HOT PICKS (Trade Immediately - High Conviction)
HOT_PICKS = [
    'DGNX',  # +21% today, +293% revenue, $9.62 (Turnaround Momentum)
    'ELWS',  # +145% revenue, 0.2s blockchain (Pre-explosive)
    'PALI',  # $1.73→$12 target = 593% upside (Biotech Catapult)
]

# ⚡ IMMEDIATE ALPHA (Your Hot List - Super Volatile)
IMMEDIATE_ALPHA = [
    'SERV', 'SGBX', 'RR', 'IONQ', 'MVST', 'TLRY', 'MU', 'SMR', 'LEU'
]

# 🧠 QUANTUM & AI (The Future - High Beta)
QUANTUM_AI = [
    'RGTI', 'QUBT', 'PLTR', 'SOUN', 'BBAI', 'SYM', 'AI', 'IONQ'
]

# 🚀 SPACE & SATELLITE (Binary Outcomes - $0 or $100)
SPACE = [
    'RKLB', 'ASTS', 'LUNR', 'SPCE', 'PL', 'BKSY', 'SPIR', 'ACHR', 'JOBY'
]

# 💰 CRYPTO & FINTECH (BTC Correlation - Liquidity Pump)
CRYPTO_FINTECH = [
    'MSTR', 'COIN', 'MARA', 'RIOT', 'CLSK', 'SOFI', 'UPST', 'AFRM', 'HOOD'
]

# ⚡ ENERGY & NUCLEAR (Power Grid Megatrend)
ENERGY_NUCLEAR = [
    'OKLO', 'NNE', 'FLNC', 'SHLS', 'SMR', 'LEU', 'PLUG', 'BE', 'QS'
]

# 🧬 BIOTECH (Catalyst-Driven - FDA Binary Events)
BIOTECH = [
    'CRSP', 'EDIT', 'NTLA', 'BEAM', 'VKTX', 'PALI', 'XBIO', 'RARE'
]

# 💻 HIGH BETA TECH (EV, Semis, Consumer)
TECH = [
    'TSLA', 'LAZR', 'MBLY', 'INVZ', 'MVST', 'MU', 'DUOL', 'RBLX', 'DKNG'
]

# 🎯 COMBINE INTO ULTIMATE UNIVERSE (Deduplicated)
TICKERS = list(set(
    PORTFOLIO + HOT_PICKS + IMMEDIATE_ALPHA + QUANTUM_AI + 
    SPACE + CRYPTO_FINTECH + ENERGY_NUCLEAR + BIOTECH + TECH
))

print(f"\n{'='*70}")
print(f"🏆 HEDGEFUND KILLER - ULTIMATE ALPHA UNIVERSE")
print(f"{'='*70}")
print(f"Total Tickers: {len(TICKERS)}")
print(f"\n  💼 Your Portfolio: {len(PORTFOLIO)} (Current Holdings)")
print(f"  🔥 Hot Picks: {len(HOT_PICKS)} (Perplexity-Analyzed - IMMEDIATE ALPHA)")
print(f"  ⚡ Immediate Alpha: {len(IMMEDIATE_ALPHA)} (Super Volatile)")
print(f"  🧠 Quantum/AI: {len(QUANTUM_AI)}")
print(f"  🚀 Space: {len(SPACE)}")
print(f"  💰 Crypto/Fintech: {len(CRYPTO_FINTECH)}")
print(f"  ⚡ Energy/Nuclear: {len(ENERGY_NUCLEAR)}")
print(f"  🧬 Biotech: {len(BIOTECH)}")
print(f"  💻 High Beta Tech: {len(TECH)}")
print(f"\n🎯 Hot Picks Details:")
print(f"  • DGNX: +21% today, +293% revenue (Turnaround Momentum)")
print(f"  • ELWS: +145% revenue, 0.2s blockchain (Pre-explosive)")
print(f"  • PALI: $1.73→$12 target = 593% upside (Biotech Catalyst)")
print(f"\n🏆 MISSION: Embarrass hedge fund billionaires in 6-12 months")
print(f"{'='*70}\n")

harvester = DataHarvester(TICKERS)
raw_data = harvester.harvest()

print(f"\n📊 Data Range: {raw_data['Datetime'].min()} to {raw_data['Datetime'].max()}")

## 🧠 MODULE 2: FEATURE ENGINE
ATR-scaled triple barrier labels + volatility-adaptive indicators.

In [ ]:
# ==========================================
# MODULE 2: FEATURE ENGINE
# ==========================================

def calculate_atr(high, low, close, period=14):
    """Manual ATR calculation"""
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return tr.rolling(period).mean()

def calculate_rsi(close, period=14):
    """Manual RSI calculation"""
    delta = close.diff()
    gain = (delta.where(delta > 0, 0)).rolling(period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(period).mean()
    rs = gain / (loss + 1e-9)
    return 100 - (100 / (1 + rs))

class FeatureEngine:
    """Production feature engineering"""
    
    def engineer(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        
        if len(df) < 200:
            return pd.DataFrame()
        
        # ATR foundation
        if HAS_PANDAS_TA:
            df['ATR_14'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)
        else:
            df['ATR_14'] = calculate_atr(df['High'], df['Low'], df['Close'])
        
        df['ATR_pct'] = df['ATR_14'] / df['Close']
        
        # Microstructure
        df['feat_spread'] = (df['High'] - df['Low']) / df['Close']
        df['feat_vol_surge'] = df['Volume'] / (df['Volume'].rolling(20).mean() + 1e-9)
        
        # RSI (adaptive)
        vol_20 = df['Close'].rolling(20).std() / df['Close']
        is_volatile = vol_20 > vol_20.quantile(0.75)
        
        if HAS_PANDAS_TA:
            rsi_fast = ta.rsi(df['Close'], length=7)
            rsi_norm = ta.rsi(df['Close'], length=14)
        else:
            rsi_fast = calculate_rsi(df['Close'], 7)
            rsi_norm = calculate_rsi(df['Close'], 14)
        
        df['feat_rsi'] = np.where(is_volatile, rsi_fast, rsi_norm)
        
        # Trend
        df['feat_sma_20'] = df['Close'].rolling(20).mean()
        df['feat_sma_50'] = df['Close'].rolling(50).mean()
        df['feat_ema_12'] = df['Close'].ewm(span=12).mean()
        df['feat_price_above_sma20'] = (df['Close'] > df['feat_sma_20']).astype(int)
        
        # Volume
        df['feat_vol_mean'] = df['Volume'].rolling(20).mean()
        df['feat_vol_trend'] = df['Volume'].rolling(5).mean() / (df['Volume'].rolling(20).mean() + 1)
        
        # Returns
        df['feat_return_1h'] = df['Close'].pct_change(1)
        df['feat_return_4h'] = df['Close'].pct_change(4)
        df['feat_return_24h'] = df['Close'].pct_change(24)
        
        # ATR-SCALED TRIPLE BARRIER TARGET
        df['upper_barrier'] = df['Close'] + (1.5 * df['ATR_14'])
        df['lower_barrier'] = df['Close'] - (1.0 * df['ATR_14'])
        
        indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=5)
        df['fwd_max'] = df['High'].rolling(window=indexer).max()
        df['fwd_min'] = df['Low'].rolling(window=indexer).min()
        
        df['target'] = ((df['fwd_max'] >= df['upper_barrier']) & 
                        (df['fwd_min'] > df['lower_barrier'])).astype(int)
        
        # 🔥 PERPLEXITY HOT PICK FEATURES (Thesis-Specific Alpha)
        ticker = df['ticker'].iloc[0] if 'ticker' in df.columns else None
        
        if ticker == 'DGNX':
            # Turnaround Momentum: Track recovery from $0.45 lows
            df['feat_recovery_pct'] = (df['Close'] - 0.45) / 0.45  # 2,037% at $9.62
            df['feat_consolidation'] = (df['Close'].rolling(5).std() / df['Close']) < 0.02
            
        elif ticker == 'ELWS':
            # Pre-explosive: Track coiling pattern
            df['feat_vol_compression'] = df['Close'].rolling(20).std() / df['Close']
            df['feat_vol_spike_ready'] = (df['Volume'].rolling(5).mean() / 
                                           df['Volume'].rolling(20).mean()) > 1.2
            
        elif ticker == 'PALI':
            # Biotech Catalyst: Track dormant → explosive
            df['feat_rsi_neutral'] = ((df['feat_rsi'] > 40) & 
                                       (df['feat_rsi'] < 60)).astype(int)
            df['feat_analyst_gap'] = 12.00 / df['Close']  # Distance to $12 target
        
        return df.dropna()

logger.info("\n🧠 Engineering features...")

feature_engine = FeatureEngine()
all_features = []

for ticker in raw_data['ticker'].unique():
    ticker_df = raw_data[raw_data['ticker'] == ticker].copy()
    features_df = feature_engine.engineer(ticker_df)
    
    if not features_df.empty:
        all_features.append(features_df)
        logger.info(f"  ✅ {ticker}: {len(features_df)} rows, {features_df['target'].mean():.1%} BUY")

full_df = pd.concat(all_features, ignore_index=True)

print(f"\n✅ Features: {len(full_df):,} rows")
print(f"   BUY signals: {full_df['target'].sum():,} ({full_df['target'].mean():.1%})")

## 🤖 MODULE 3-4: REGIME + ENSEMBLE TRAINING
IWM regime detection + XGBoost GPU training with walk-forward validation.

In [ ]:
# ==========================================
# MODULES 3-4: REGIME + ENSEMBLE
# ==========================================

class RegimeClassifier:
    def classify_regime(self) -> str:
        try:
            iwm = yf.download('IWM', period='30d', interval='1d', progress=False)
            vix_data = yf.download('^VIX', period='5d', interval='1d', progress=False)
            
            iwm_return = (iwm['Close'].iloc[-1] / iwm['Close'].iloc[-21] - 1) if len(iwm) >= 21 else 0
            vix = vix_data['Close'].iloc[-1] if not vix_data.empty else 20
            
            logger.info(f"📊 IWM: {iwm_return:.2%} | VIX: {vix:.1f}")
            
            if vix > 30:
                return 'PANIC'
            elif iwm_return > 0.05:
                return 'BULL_LOW' if vix < 15 else 'BULL_MID'
            elif iwm_return < -0.05:
                return 'BEAR_LOW' if vix < 15 else 'BEAR_MID'
            else:
                return 'CHOPPY'
        except:
            return 'CHOPPY'

regime_classifier = RegimeClassifier()
current_regime = regime_classifier.classify_regime()
print(f"\n🌍 Regime: {current_regime}")

# ENSEMBLE TRAINING
class EnsembleClassifier:
    def __init__(self):
        self.results = []
        
    def train(self, X, y, n_splits=3):
        features = [c for c in X.columns if 'feat_' in c]
        X_features = X[features].copy()
        
        tscv = TimeSeriesSplit(n_splits=n_splits)
        
        for fold, (train_idx, test_idx) in enumerate(tscv.split(X_features), 1):
            logger.info(f"\n🔄 FOLD {fold}/{n_splits}")
            
            X_train, X_test = X_features.iloc[train_idx], X_features.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            
            # Scale
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
            # Train
            scale_pos_weight = (y_train == 0).sum() / max((y_train == 1).sum(), 1)
            
            model = xgb.XGBClassifier(
                n_estimators=2000,
                learning_rate=0.005,
                max_depth=7,
                subsample=0.7,
                colsample_bytree=0.7,
                scale_pos_weight=scale_pos_weight,
                tree_method='hist',
                device='cuda' if 'torch' in dir() and torch.cuda.is_available() else 'cpu',
                early_stopping_rounds=100,
                random_state=42
            )
            
            model.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)], verbose=False)
            
            # Evaluate
            preds = model.predict(X_test_scaled)
            precision = precision_score(y_test, preds)
            f1 = f1_score(y_test, preds)
            auc = roc_auc_score(y_test, model.predict_proba(X_test_scaled)[:, 1])
            
            logger.info(f"  Precision: {precision:.2%} | F1: {f1:.2%} | AUC: {auc:.2%}")
            
            self.results.append({
                'fold': fold,
                'precision': precision,
                'f1': f1,
                'auc': auc,
                'model': model,
                'scaler': scaler,
                'features': features
            })
        
        avg_precision = np.mean([r['precision'] for r in self.results])
        logger.info(f"\n🏆 Avg Precision: {avg_precision:.2%}")
        
        return self.results

ensemble = EnsembleClassifier()
features_for_training = [c for c in full_df.columns if 'feat_' in c]
X_train = full_df[features_for_training]
y_train = full_df['target']

training_results = ensemble.train(X_train, y_train)

best_result = max(training_results, key=lambda x: x['precision'])
print(f"\n💾 Best: Fold {best_result['fold']}, {best_result['precision']:.2%} precision")

## 💾 SAVE PRODUCTION ARTIFACTS
Export trained models for deployment.

In [ ]:
# ==========================================
# SAVE MODELS
# ==========================================

logger.info("\n💾 Saving artifacts...")

joblib.dump(best_result['model'], 'alpha76_model.pkl')
joblib.dump(best_result['scaler'], 'alpha76_scaler.pkl')

metadata = {
    'training_date': datetime.now().isoformat(),
    'tickers': TICKERS,
    'avg_precision': float(np.mean([r['precision'] for r in training_results])),
    'avg_f1': float(np.mean([r['f1'] for r in training_results])),
    'features': best_result['features'],
    'current_regime': current_regime
}

with open('alpha76_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("\n" + "="*70)
print("🏆 TRAINING COMPLETE")
print("="*70)
print(f"Precision: {metadata['avg_precision']:.2%}")
print(f"F1 Score:  {metadata['avg_f1']:.2%}")
print(f"Files: alpha76_model.pkl, alpha76_scaler.pkl, alpha76_metadata.json")
print("\n🚂 READY FOR DEPLOYMENT!")